# LIGO Black Holes

A real-science demo using Numpy, SciPy, and Pandas

LIGO presented their data analysis in an [interactive Jupyter notebook](https://github.com/minrk/ligo-binder) using Numpy, SciPy, Matplotlib, and HDF5. This notebook is a simplification of that analysis, in which I've added Pandas and Vega plotting.

Pandas reduced the date-time and plotting code by about 10 lines → 1 line.

The signal processing is over my head; I just copied it.

We'll need a few more libraries for this one:

In [ ]:
!pip install h5py pdvega scipy --user

In [ ]:
!pip install matplotlib==2.0.2 --user

In [ ]:
# Python standard library
import json
import datetime

# scientific Python ecosystem
import numpy
import pandas
import pdvega
import scipy.interpolate   # SciPy is a bag of classic algorithms
import scipy.signal        # including signal processing
import matplotlib.mlab     # some signal processing in Matplotlib (poor separation of concerns...)
import h5py

# LIGO-specific library for interpreting their HDF5 files; like "fwlite"
import readligo

All of the plots can be made with Matplotlib (blurry but fast: it's just PNGs in notebook) or Vega (sharp and interactive: it's Javascript). To use Matplotlib, you need to execute this magic:

In [ ]:
%matplotlib inline

LIGO distributes their data as arrays in HDF5 files accompanied by metadata in JSON file for interpreting them.

In [ ]:
events = json.load(open("LIGO/BBH_events_v3.json"))

The JSON files describe multiple events. Let's take the one from Jan 4, 2017.

In [ ]:
event = events["GW170104"]
event

Load the array data. H1 is a 4 km interferometer at Hanford, Washington, and L1 is a 4 km interferometer at Livingston Parish, Louisiana. The data are indexed by time.

In [ ]:
strain_H1, time_H1, chan_dict_H1 = readligo.loaddata("LIGO/" + event["fn_H1"], "H1")
strain_L1, time_L1, chan_dict_L1 = readligo.loaddata("LIGO/" + event["fn_L1"], "L1")

df_H1 = pandas.DataFrame(index=time_H1, data={"H1": strain_H1})
df_L1 = pandas.DataFrame(index=time_L1, data={"L1": strain_L1})

Pandas is for manipulating data with different indexes. If LIGO had used Pandas, they wouldn't have had to ensure that `time_H1 == time_L1`. You can just combine them like this:

In [ ]:
df = pandas.concat([df_H1, df_L1], axis=1)
df

The time index is the number of seconds since some date that is not 1970 (the UNIX standard). Fortunately, LIGO provided a Rosetta stone: a single timestamp as text and as a number.

In [ ]:
event["tevent"], event["utcevent"]

Use Pandas tools to turn the index into `datetimes`.

In [ ]:
originutc = pandas.to_datetime(event["utcevent"])
originsec = pandas.to_datetime(event["tevent"], unit="s")
originutc - originsec

In [ ]:
time_offset = (originutc - originsec).total_seconds()
df.index = pandas.to_datetime(df.index + time_offset, unit="s")
df

The data are 30 seconds wide, but the signal is only a fraction of a second around the origin.

In [ ]:
df.index.max() - df.index.min()

We make filters (Numpy arrays of booleans), just as in the Numpy notebook.

In [ ]:
wide_window = numpy.logical_and(
    originutc - datetime.timedelta(seconds=5) <= df.index,
    df.index < originutc + datetime.timedelta(seconds=5))

narrow_window = numpy.logical_and(
    originutc - datetime.timedelta(seconds=0.15) <= df.index,
    df.index < originutc + datetime.timedelta(seconds=0.05))

wide_window[:len(wide_window) // 2]

Now let's plot it in this window. Vega is a little slow (turn off interactivity), but Matplotlib is blurry.

In [ ]:
df[wide_window].vgplot.line(interactive=False)
# df[wide_window].plot.line()

The signal is deeply buried in that plot. To bring it out, they use signal processing. I'm not going to pretend to understand exactly what these filters do.

In [ ]:
Pxx_H1, freqs = matplotlib.mlab.psd(strain_H1, Fs=event["fs"], NFFT=4*event["fs"])
Pxx_L1, freqs = matplotlib.mlab.psd(strain_L1, Fs=event["fs"], NFFT=4*event["fs"])

psd_H1 = scipy.interpolate.interp1d(freqs, Pxx_H1)
psd_L1 = scipy.interpolate.interp1d(freqs, Pxx_L1)

dt = time_H1[1] - time_H1[0]   # uniformly sampled

def whiten(strain, interp_psd):
    Nt = len(strain)
    freqs = numpy.fft.rfftfreq(Nt, dt)
    freqs1 = numpy.linspace(0, 2048, Nt // 2 + 1)
    hf = numpy.fft.rfft(strain)
    white_hf = hf / (numpy.sqrt(interp_psd(freqs) / dt / 2.0))
    white_ht = numpy.fft.irfft(white_hf, n=Nt)
    return white_ht

strain_H1_whiten = whiten(strain_H1, psd_H1)
strain_L1_whiten = whiten(strain_L1, psd_L1)

bb, ab = scipy.signal.butter(
    4, [event["fband"][0]*2.0/event["fs"],
        event["fband"][1]*2.0/event["fs"]], btype="band")

strain_H1_whitenbp = scipy.signal.filtfilt(bb, ab, strain_H1_whiten)
strain_L1_whitenbp = scipy.signal.filtfilt(bb, ab, strain_L1_whiten)

Add these new arrays to the data frame (with the same index).

In [ ]:
df["L1 whitened"] = pandas.Series(strain_L1_whitenbp, index=df.index)
df["H1 whitened"] = pandas.Series(strain_H1_whitenbp, index=df.index)
df

And plot them. I can see the black hole! Can you?

In [ ]:
df[["L1 whitened", "H1 whitened"]][narrow_window].vgplot.line()

Before moving on, we have to do something about the date axis on that plot. It's useless. Let's change it to seconds before and after the origin.

In [ ]:
df2 = df.copy()
df2.index = (df2.index - originutc).total_seconds()
df2

In [ ]:
df2.index.name = "seconds before or after origin"
df2[["L1 whitened", "H1 whitened"]][narrow_window].vgplot.line(
    value_name="whitened strain (units of noise stdev)", var_name="source", width=800, height=400)

Let's add a template (simulation) to overlay on the data. This looks daunting until you realize that it's the entirety of their Monte Carlo.

In [ ]:
# read in the template (plus and cross) and parameters for the theoretical waveform
f_template = h5py.File("LIGO/" + event["fn_template"], "r")

# extract metadata from the template file:
template_p, template_c = f_template["template"][...]
t_m1 = f_template["/meta"].attrs["m1"]
t_m2 = f_template["/meta"].attrs["m2"]
t_a1 = f_template["/meta"].attrs["a1"]
t_a2 = f_template["/meta"].attrs["a2"]
t_approx = f_template["/meta"].attrs["approx"]
f_template.close()

# the template extends to roughly 16s, zero-padded to the 32s data length. The merger will be roughly 16s in.
template_offset = 16.0

# whiten the templates:
template_p_whiten = whiten(template_p, psd_H1)
template_c_whiten = whiten(template_c, psd_H1)
template_p_whitenbp = scipy.signal.filtfilt(bb, ab, template_p_whiten)
template_c_whitenbp = scipy.signal.filtfilt(bb, ab, template_c_whiten)

# constants:
clight = 2.99792458e8                # m/s
G = 6.67259e-11                      # m^3/kg/s^2 
MSol = 1.989e30                      # kg

# template parameters: masses in units of MSol:
t_mtot = t_m1 + t_m2
# final BH mass is typically 95% of the total initial mass:
t_mfin = t_mtot*0.95
# Final BH radius, in km:
R_fin = 2*G*t_mfin*MSol/clight**2/1000.0

# complex template:
template = (template_p + template_c*10.j)
ttime = time_H1 - time_H1[0] - template_offset

# compute the instantaneous frequency of this chirp-like signal:
tphase = numpy.unwrap(numpy.angle(template))
fGW = numpy.gradient(tphase)*event["fs"]/(2.0*numpy.pi)
# fix discontinuities at the very end:
# iffix = numpy.where(numpy.abs(numpy.gradient(fGW)) > 100.)[0]
iffix = numpy.where(numpy.abs(template) < numpy.abs(template).max()*0.001)[0]
fGW[iffix] = fGW[iffix[0] - 1]
fGW[numpy.where(fGW < 1.0)] = fGW[iffix[0] - 1]

# compute v/c:
voverc = (G*t_mtot*MSol*numpy.pi*fGW/clight**3)**(1.0/3.0)

# index where f_GW is in-band:
f_inband = event["fband"][0]
iband = numpy.where(fGW > f_inband)[0][0]
# index at the peak of the waveform:
ipeak = numpy.argmax(numpy.abs(template))

# number of cycles between inband and peak:
Ncycles = (tphase[ipeak] - tphase[iband])/(2.0*numpy.pi)

# -- To calculate the PSD of the data, choose an overlap and a window (common to all detectors)
#   that minimizes "spectral leakage" https://en.wikipedia.org/wiki/Spectral_leakage
NFFT = 4*event["fs"]
psd_window = numpy.blackman(NFFT)
# and a 50% overlap:
NOVL = NFFT/2

# define the complex template, common to both detectors:
template = (template_p + template_c*1.0j)
# We will record the time where the data match the END of the template.
etime = time_H1 + template_offset
# the length and sampling rate of the template MUST match that of the data.
datafreq = numpy.fft.fftfreq(template.size)*event["fs"]

# to remove effects at the beginning and end of the data stretch, window the data
# https://en.wikipedia.org/wiki/Window_function#Tukey_window
try:
    # Tukey window preferred, but requires recent scipy version
    dwindow = scipy.signal.tukey(template.size, alpha=1.0/8)
except:
    # Blackman window OK if Tukey is not available
    dwindow = scipy.signal.blackman(template.size)

# prepare the template fft.
template_fft = numpy.fft.fft(template*dwindow) / event["fs"]

data = strain_L1.copy()

data_psd, freqs = matplotlib.mlab.psd(data, Fs=event["fs"], NFFT=NFFT, window=psd_window, noverlap=NOVL)

# Take the Fourier Transform (FFT) of the data and the template (with dwindow)
data_fft = numpy.fft.fft(data*dwindow) / event["fs"]

# -- Interpolate to get the PSD values at the needed frequencies
power_vec = numpy.interp(numpy.abs(datafreq), freqs, data_psd)

# -- Calculate the matched filter output in the time domain:
# Multiply the Fourier Space template and data, and divide by the noise power in each frequency bin.
# Taking the Inverse Fourier Transform (IFFT) of the filter output puts it back in the time domain,
# so the result will be plotted as a function of time off-set between the template and the data:
optimal = data_fft * template_fft.conjugate() / power_vec
optimal_time = 2*numpy.fft.ifft(optimal)*event["fs"]

# -- Normalize the matched filter output:
# Normalize the matched filter output so that we expect a value of 1 at times of just noise.
# Then, the peak of the matched filter output will tell us the signal-to-noise ratio (SNR) of the signal.
sigmasq = 1*(template_fft * template_fft.conjugate() / power_vec).sum() * numpy.abs(datafreq[1] - datafreq[0])
sigma = numpy.sqrt(numpy.abs(sigmasq))
SNR_complex = optimal_time/sigma

# shift the SNR vector by the template length so that the peak is at the END of the template
peaksample = int(data.size / 2)  # location of peak in the template
SNR_complex = numpy.roll(SNR_complex, peaksample)
SNR = abs(SNR_complex)

# find the time and SNR value at maximum:
indmax = numpy.argmax(SNR)
timemax = time_H1[indmax]
SNRmax = SNR[indmax]

# Calculate the "effective distance" (see FINDCHIRP paper for definition)
# d_eff = (8. / SNRmax)*D_thresh
d_eff = sigma / SNRmax
# -- Calculate optimal horizon distnace
horizon = sigma / 8

# Extract time offset and phase at peak
phase = numpy.angle(SNR_complex[indmax])
offset = (indmax-peaksample)

# apply time offset, phase, and d_eff to whitened template, for plotting
template_whitened = (template_p_whitenbp + template_c_whitenbp*1.0j) 
template_phaseshifted = numpy.real(template_whitened*numpy.exp(1j*phase))
template_match = numpy.roll(template_phaseshifted,offset) / d_eff

Now add it to the data frame and plot. Perfect!

In [ ]:
df2["template"] = pandas.Series(template_match, index=df2.index)

In [ ]:
df2[["L1 whitened", "H1 whitened", "template"]][narrow_window].vgplot.line(
    value_name="whitened strain (units of noise stdev)", var_name="source", width=1000, height=500)